## Imports

In [18]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.callbacks import LearningRateScheduler

In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [3]:
%cd /content/gdrive/MyDrive/CS_2

/content/gdrive/MyDrive/CS_2


# Word_level

### Data

In [6]:
data=pd.read_csv('data_wl')

In [7]:
tr,val = train_test_split(data, test_size=0.01) # as we have very less data to work with, we use major portion to train the data.

In [8]:
# Creating the word tokens from the sentences
tokenizer_raw = Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n', lower=True)
tokenizer_raw.fit_on_texts(tr['Source'].values)

tokenizer_std = Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n', lower=True)
tokenizer_std.fit_on_texts(tr['Target_inp'].values)

In [9]:
Source_vocab_size=len(tokenizer_raw.word_index.keys())+1 
Target_vocab_size=len(tokenizer_std.word_index.keys())+1

In [10]:
# Assigning token digits to every word in a sentence
Source_seq_tr = tokenizer_raw.texts_to_sequences(tr['Source']) 
Target_inp_seq_tr = tokenizer_std.texts_to_sequences(tr['Target_inp'])
Target_out_seq_tr = tokenizer_std.texts_to_sequences(tr['Target_out'])

Source_seq_val = tokenizer_raw.texts_to_sequences(val['Source'])
Target_inp_seq_val = tokenizer_std.texts_to_sequences(val['Target_inp'])
Target_out_seq_val = tokenizer_std.texts_to_sequences(val['Target_out'])

In [11]:
l=[]
for i in Source_seq_tr:
    l.append(len(i))
max_len=max(l)

In [12]:
l=[]
for i in Target_inp_seq_tr:
    l.append(len(i))
max_len_dec_ip=max(l)

l=[]
for i in Target_out_seq_tr:
    l.append(len(i))
max_len_dec_op=max(l)

max_len_dec=max(max_len_dec_ip,max_len_dec_op)

In [13]:
#Padding the sentences with zeros so that all sentences are of equal length
target_inp_tr = pad_sequences(Target_inp_seq_tr,  maxlen=max_len_dec, padding='post')
target_out_tr = pad_sequences(Target_out_seq_tr, maxlen=max_len_dec, padding='post')
Source_inp_tr = pad_sequences(Source_seq_tr, maxlen=max_len, padding='post')

target_inp_val = pad_sequences(Target_inp_seq_val,  maxlen=max_len_dec, padding='post')
target_out_val = pad_sequences(Target_out_seq_val, maxlen=max_len_dec, padding='post')
Source_inp_val = pad_sequences(Source_seq_val, maxlen=max_len, padding='post')

### Custom Functions

In [14]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, vocab_size, embedding_dim, enc_units,input_length):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.input_length = input_length
        self.enc_units= enc_units
        self.lstm_output = 0
        self.lstm_state_h=0
        self.lstm_state_c=0
        
    def build(self, input_shape):
        self.embedding = Embedding(input_dim=self.vocab_size, output_dim=self.embedding_dim,trainable=True,input_length=self.input_length,
                           mask_zero=True, name="embedding_layer_encoder")
        self.lstm = LSTM(self.enc_units, return_state=True, return_sequences=True, name="Encoder_LSTM")
        
    def call(self, input_sentances, training=True):
        input_embedd                           = self.embedding(input_sentances)
        self.lstm_output, self.lstm_state_h,self.lstm_state_c = self.lstm(input_embedd)
        return self.lstm_output, self.lstm_state_h,self.lstm_state_c

    def initialize_states(self, batch_size):
        return tf.zeros((batch_size, self.enc_units)), tf.zeros((batch_size, self.enc_units))

    def get_states(self):
        return self.lstm_state_h,self.lstm_state_c

In [15]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, vocab_size, embedding_dim,dec_units, input_length):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.dec_units = dec_units
        self.input_length = input_length
        self.embedding = Embedding(input_dim=self.vocab_size, output_dim=self.embedding_dim,input_length=self.input_length,
                           mask_zero=True, name="embedding_layer_decoder",trainable=True)
        self.lstm = LSTM(self.dec_units, return_sequences=True, return_state=True, name="Encoder_LSTM")
    
    def call(self, target_sentances,states):
        target_embedd           = self.embedding(target_sentances)
        lstm_output,decoder_final_state_h,decoder_final_state_c      = self.lstm(target_embedd, initial_state=states)
        return  lstm_output,decoder_final_state_h,decoder_final_state_c 

In [16]:
class Encoder_decoder(Model):
    def __init__(self, encoder_inputs_length,decoder_inputs_length, output_vocab_size,batch_size):
        super().__init__() # https://stackoverflow.com/a/27134600/4084039
        self.encoder = Encoder(vocab_size=Source_vocab_size, embedding_dim=50, input_length=encoder_inputs_length, enc_units=256)
        self.decoder = Decoder(vocab_size=Target_vocab_size, embedding_dim=100, input_length=decoder_inputs_length, dec_units=256)
        self.dense   = Dense(output_vocab_size)
        self.batch_size=batch_size
        
    def call(self, data):
        input,output = data[0], data[1]
        enc_initial_states = self.encoder.initialize_states(self.batch_size)
        encoder_output, encoder_h, encoder_c = self.encoder(input,enc_initial_states)
        states=[encoder_h, encoder_c]
        decoder_output,decoder_final_state_h,decoder_final_state_c    = self.decoder(output,states)
        output                               = self.dense(decoder_output)
        return output

In [17]:
model  = Encoder_decoder(encoder_inputs_length=max_len,decoder_inputs_length=max_len_dec,output_vocab_size=Source_vocab_size, batch_size=256)

In [21]:
import os
import datetime
checkpoint_1 = keras.callbacks.ModelCheckpoint('weights', save_best_only= True, monitor='val_loss', mode = 'min', verbose= 1)
tensorboard_callback = keras.callbacks.TensorBoard('log', histogram_freq=0)

# Reduce learning rate based on the validation loss
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1, mode="min", verbose=1)

In [24]:
def custom_lossfunction(real, pred):
    #https://www.tensorflow.org/tutorials/text/nmt_with_attention#define_the_optimizer_and_the_loss_function
    loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')    
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)

optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer,loss=custom_lossfunction)

### Training

In [ ]:
history=model.fit([Source_inp_tr, target_inp_tr],target_out_tr ,epochs=50,batch_size=256,
                  validation_data=([Source_inp_val, target_inp_val], target_out_val),callbacks=[reduce_lr, checkpoint_1, tensorboard_callback])

Epoch 1/50
8/8 [==============================] - 10s 477ms/step - loss: 0.9072 - val_loss: 0.6388

Epoch 00001: val_loss improved from inf to 0.63883, saving model to weights
Epoch 2/50
8/8 [==============================] - 0s 45ms/step - loss: 0.8805 - val_loss: 0.5744

Epoch 00002: val_loss improved from 0.63883 to 0.57441, saving model to weights
Epoch 3/50
8/8 [==============================] - 0s 45ms/step - loss: 0.7616 - val_loss: 0.4779

Epoch 00003: val_loss improved from 0.57441 to 0.47789, saving model to weights
Epoch 4/50
8/8 [==============================] - 0s 45ms/step - loss: 0.6929 - val_loss: 0.4701

Epoch 00004: val_loss improved from 0.47789 to 0.47007, saving model to weights
Epoch 5/50
8/8 [==============================] - 0s 44ms/step - loss: 0.6706 - val_loss: 0.4681

Epoch 00005: val_loss improved from 0.47007 to 0.46810, saving model to weights
Epoch 6/50
8/8 [==============================] - 0s 46ms/step - loss: 0.6728 - val_loss: 0.4661

Epoch 00006: v

In [ ]:
def predict(input_sentence, model):
    input_tokens = tokenizer_raw.texts_to_sequences([input_sentence])
    input_sequence = pad_sequences(input_tokens, maxlen=max_len, padding='post')

  # Getting encoder output and states
    enc_initial_states = model.encoder.initialize_states(len(input_sequence))
    enc_out, enc_state_h, enc_state_c = model.encoder(input_sequence, enc_initial_states)
    state_h, state_c = enc_state_h, enc_state_c
    states=[state_h, state_c]
    target_word = np.zeros((1,1))
    target_word[0,0] = tokenizer_std.word_index['<start>']
    stop_condition = False
    decoded_sent = ''
    pos = 0
    while not stop_condition:
    # decoder layer, intial states are encoder's final states
        output_dec,dec_state_h,dec_state_c =model.layers[1](target_word,states)
        output=model.layers[2](output_dec[0])
        predicted_id = tf.argmax(output[0]).numpy()
        decoded_sent += tokenizer_std.index_word[predicted_id] + ' '
        
        
        # Get all the outputs till we don't get '<end>' word or reached maximum output length
        if tokenizer_std.index_word[predicted_id] == '<end>' or len(decoded_sent.split()) >= 20:
            stop_condition = True

    # current output word is input word for next timestamp
        target_word = np.zeros((1,1))
        target_word[0,0] = predicted_id

    # current out states are input states for next timestamp
        state_h, state_c = dec_state_h, dec_state_c
        states=[state_h, state_c]

        pos += 1

    return decoded_sent

### Predictions

In [ ]:
for i in range(0,5):
    input_sentence=tr.Source.iloc[i]
    print('Input:',input_sentence)
    print('Prediction:',predict(input_sentence, model))
    print('Actual:',tr.Target_out.iloc[i])
    print('---------------------------------------')

Input: Wad u want to major?
Prediction: i you you you you you you you you you you you you you you you you you you you 
Actual: What do you want to major? <end>
---------------------------------------
Input: Yest we try until so long... Today i go sch they oso cant login... Waste our time yest...
Prediction: i i you you you you you you you you you you you you you you you you you you 
Actual: Yesterday we try until so long. Today I go to school, they also can't login. Waste our time yesterday. <end>
---------------------------------------
Input: They inside e restaurant liao.
Prediction: i you you you you you you you you you you you you you you you you you you you 
Actual: They are inside the restaurant. <end>
---------------------------------------
Input: Ok,meet you there at 8.45pm, because no taxi and it was raining
Prediction: i i you you you you you you you you you you you you you you you you you you 
Actual: Ok, meet you there at 8:45pm, because no taxi and it was raining. <end>
--

In [ ]:
loss=model.evaluate([Source_inp_val, target_inp_val], target_out_val)

1/1 [==============================] - 0s 19ms/step - loss: 0.4579


### Perplexity

<font color='green'>Perplexity is the measure of how well a model predicts the sample. It is defined as nth root over the inverse probabilties of a sentence. 

<font color='green'>Predicting higher probability for the actual sentences denotes higher performance of the model, as the perplexity is inverse 0f probaiblity, lower the perplexity better the model.

<font color='green'>Mathematically, the perplexity can be derived as the exponential of the cross entropy. as the loass we used in our model is crossentropy, we calculate 2^loss to get the perplexity value.

In [ ]:
#https://towardsdatascience.com/perplexity-in-language-models-87a196019a94
perplexity=2**(loss)
perplexity

1.3735523329934292

### Bleu Score

<font color='green'>Bleu Score is also a metric for evaluating a generated sentence to a reference sentence. Bleu score ranges from 0 to 1. Higher the score, better the model.

In [ ]:
import nltk.translate.bleu_score as bleu
bleu_score=[]
for i in range(20):
  decoded_sent=predict(val.Source.values[i].split(), model)
  bleu_score.append(bleu.sentence_bleu(val.Target_out.values[i].split(), decoded_sent))

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [ ]:
avg_bleu_score=sum(bleu_score)/len(bleu_score)
avg_bleu_score

0.24927488041111542

# Character_level

### Data

In [ ]:
data=pd.read_csv('data_cl')

In [ ]:
data

,Source,Target_inp,Target_out
0,"U wan me to ""chop"" seat 4 u nt?",\t Do you want me to reserve seat for you or not?,Do you want me to reserve seat for you or not?\n
1,Yup. U reaching. We order some durian pastry a...,\t Yeap. You reaching? We ordered some Durian ...,Yeap. You reaching? We ordered some Durian pas...
2,They become more ex oredi... Mine is like 25.....,\t They become more expensive already. Mine is...,They become more expensive already. Mine is li...
3,I'm thai. what do u do?,\t I'm Thai. What do you do?,I'm Thai. What do you do?\n
4,Hi! How did your week go? Haven heard from you...,\t Hi! How did your week go? Haven't heard fro...,Hi! How did your week go? Haven't heard from y...
...,...,...,...
1988,Hi tina ...,\t Hi tina.,Hi tina.\n
1989,Hmmm... Thk i usually book on wkends... Depend...,\t Hmm. I think I usually book on weekends. It...,Hmm. I think I usually book on weekends. It de...
1990,We r near coca oredi...,\t We are near Coca already.,We are near Coca already.\n
1991,hall Eleven. Got lectures le mah.en forget abt...,\t Hall eleven. Got lectures. And forget about...,Hall eleven. Got lectures. And forget about co...


In [ ]:
tr,val = train_test_split(data, test_size=0.05) # as we have very less data to work with, we use major portion to train the data.

In [ ]:
# Creating the word tokens from the sentences
tokenizer_raw = Tokenizer(filters=None, char_level=True, lower=False)
tokenizer_raw.fit_on_texts(tr['Source'].values)

tokenizer_std = Tokenizer(filters=None, char_level=True, lower=False)
tokenizer_std.fit_on_texts(tr['Target_inp'].values)

In [ ]:
#https://www.geeksforgeeks.org/python-string-printable/
import string
printable_chars=[]
for i in string.printable: # Gives the printable characters
    printable_chars.append(i)

In [ ]:
words = dict()
for i in range(len(printable_chars)):
  words[printable_chars[i]] = i+1

In [ ]:
tokenizer_raw.word_index=words
tokenizer_std.word_index=words

In [ ]:
index = dict()
for i in range(len(printable_chars)):
  index[i+1] = printable_chars[i]

In [ ]:
tokenizer_raw.index_word=index
tokenizer_std.index_word=index

In [ ]:
Source_vocab_size=len(tokenizer_raw.word_index.keys())+1 
Target_vocab_size=len(tokenizer_std.word_index.keys())+1

In [ ]:
# Assigning token digits to every word in a sentence
Source_seq_tr = tokenizer_raw.texts_to_sequences(tr['Source']) 
Target_inp_seq_tr = tokenizer_std.texts_to_sequences(tr['Target_inp'])
Target_out_seq_tr = tokenizer_std.texts_to_sequences(tr['Target_out'])

Source_seq_val = tokenizer_raw.texts_to_sequences(val['Source'])
Target_inp_seq_val = tokenizer_std.texts_to_sequences(val['Target_inp'])
Target_out_seq_val = tokenizer_std.texts_to_sequences(val['Target_out'])

In [ ]:
l=[]
for i in Source_seq_tr:
    l.append(len(i))
max_len=max(l)

In [ ]:
l=[]
for i in Target_inp_seq_tr:
    l.append(len(i))
max_len_dec_ip=max(l)

l=[]
for i in Target_out_seq_tr:
    l.append(len(i))
max_len_dec_op=max(l)

max_len_dec=max(max_len_dec_ip,max_len_dec_op)

In [ ]:
#Padding the sentences with zeros so that all sentences are of equal length
target_inp_tr = pad_sequences(Target_inp_seq_tr,  maxlen=max_len_dec, padding='post')
target_out_tr = pad_sequences(Target_out_seq_tr, maxlen=max_len_dec, padding='post')
Source_inp_tr = pad_sequences(Source_seq_tr, maxlen=max_len, padding='post')

target_inp_val = pad_sequences(Target_inp_seq_val,  maxlen=max_len_dec, padding='post')
target_out_val = pad_sequences(Target_out_seq_val, maxlen=max_len_dec, padding='post')
Source_inp_val = pad_sequences(Source_seq_val, maxlen=max_len, padding='post')

### Custom Functions

In [ ]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, vocab_size, embedding_dim, enc_units,input_length):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.input_length = input_length
        self.enc_units= enc_units
        self.lstm_output = 0
        self.lstm_state_h=0
        self.lstm_state_c=0
        
    def build(self, input_shape):
        self.embedding = Embedding(input_dim=self.vocab_size, output_dim=self.embedding_dim,trainable=True,input_length=self.input_length,
                           mask_zero=True, name="embedding_layer_encoder")
        self.lstm = LSTM(self.enc_units, return_state=True, return_sequences=True, name="Encoder_LSTM")
        
    def call(self, input_sentances, training=True):
        input_embedd                           = self.embedding(input_sentances)
        self.lstm_output, self.lstm_state_h,self.lstm_state_c = self.lstm(input_embedd)
        return self.lstm_output, self.lstm_state_h,self.lstm_state_c

    def initialize_states(self, batch_size):
        return tf.zeros((batch_size, self.enc_units)), tf.zeros((batch_size, self.enc_units))

    def get_states(self):
        return self.lstm_state_h,self.lstm_state_c

In [ ]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, vocab_size, embedding_dim,dec_units, input_length):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.dec_units = dec_units
        self.input_length = input_length
        self.embedding = Embedding(input_dim=self.vocab_size, output_dim=self.embedding_dim,input_length=self.input_length,
                           mask_zero=True, name="embedding_layer_decoder",trainable=True)
        self.lstm = LSTM(self.dec_units, return_sequences=True, return_state=True, name="Encoder_LSTM")
    
    def call(self, target_sentances,states):
        target_embedd           = self.embedding(target_sentances)
        lstm_output,decoder_final_state_h,decoder_final_state_c      = self.lstm(target_embedd, initial_state=states)
        return  lstm_output,decoder_final_state_h,decoder_final_state_c 

In [ ]:
class Encoder_decoder(Model):
    def __init__(self, encoder_inputs_length,decoder_inputs_length, output_vocab_size,batch_size):
        super().__init__() # https://stackoverflow.com/a/27134600/4084039
        self.encoder = Encoder(vocab_size=Source_vocab_size, embedding_dim=50, input_length=encoder_inputs_length, enc_units=256)
        self.decoder = Decoder(vocab_size=Target_vocab_size, embedding_dim=100, input_length=decoder_inputs_length, dec_units=256)
        self.dense   = Dense(output_vocab_size)
        self.batch_size=batch_size
        
    def call(self, data):
        input,output = data[0], data[1]
        enc_initial_states = self.encoder.initialize_states(self.batch_size)
        encoder_output, encoder_h, encoder_c = self.encoder(input,enc_initial_states)
        states=[encoder_h, encoder_c]
        decoder_output,decoder_final_state_h,decoder_final_state_c    = self.decoder(output,states)
        output                               = self.dense(decoder_output)
        return output

In [ ]:
model  = Encoder_decoder(encoder_inputs_length=max_len,decoder_inputs_length=max_len_dec,output_vocab_size=Source_vocab_size, batch_size=256)

In [ ]:
def custom_lossfunction(real, pred):
    #https://www.tensorflow.org/tutorials/text/nmt_with_attention#define_the_optimizer_and_the_loss_function
    loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')    
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)

optimizer = tf.keras.optimizers.Adam(lr=0.01)
model.compile(optimizer=optimizer,loss=custom_lossfunction)

In [ ]:
import os
import datetime
checkpoint_1 = keras.callbacks.ModelCheckpoint('weights', save_best_only= True, monitor='val_loss', mode = 'min', verbose= 1)
tensorboard_callback = keras.callbacks.TensorBoard('log', histogram_freq=0)

# Reduce learning rate based on the validation loss
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, mode="min", verbose=1)

### Training

In [ ]:
history=model.fit([Source_inp_tr, target_inp_tr],target_out_tr ,epochs=50,batch_size=256,
                  validation_data=([Source_inp_val, target_inp_val], target_out_val),callbacks=[reduce_lr, checkpoint_1, tensorboard_callback])

Epoch 1/50
8/8 [==============================] - 14s 745ms/step - loss: 0.4104 - val_loss: 0.3773

Epoch 00001: val_loss improved from inf to 0.37733, saving model to weights
Epoch 2/50
8/8 [==============================] - 2s 197ms/step - loss: 0.3497 - val_loss: 0.3640

Epoch 00002: val_loss improved from 0.37733 to 0.36395, saving model to weights
Epoch 3/50
8/8 [==============================] - 1s 186ms/step - loss: 0.3482 - val_loss: 0.3569

Epoch 00003: val_loss improved from 0.36395 to 0.35687, saving model to weights
Epoch 4/50
8/8 [==============================] - 1s 184ms/step - loss: 0.3353 - val_loss: 0.3513

Epoch 00004: val_loss improved from 0.35687 to 0.35130, saving model to weights
Epoch 5/50
8/8 [==============================] - 1s 187ms/step - loss: 0.3367 - val_loss: 0.3469

Epoch 00005: val_loss improved from 0.35130 to 0.34694, saving model to weights
Epoch 6/50
8/8 [==============================] - 1s 186ms/step - loss: 0.3292 - val_loss: 0.3431

Epoch 000

In [ ]:
history=model.fit([Source_inp_tr, target_inp_tr],target_out_tr ,epochs=50,batch_size=256,
                  validation_data=([Source_inp_val, target_inp_val], target_out_val),callbacks=[reduce_lr, checkpoint_1, tensorboard_callback])

Epoch 1/50
8/8 [==============================] - 3s 345ms/step - loss: 0.2072 - val_loss: 0.2762

Epoch 00001: val_loss did not improve from 0.27591
Epoch 2/50
8/8 [==============================] - 1s 185ms/step - loss: 0.2068 - val_loss: 0.2765

Epoch 00002: val_loss did not improve from 0.27591
Epoch 3/50
8/8 [==============================] - 1s 185ms/step - loss: 0.2064 - val_loss: 0.2766

Epoch 00003: val_loss did not improve from 0.27591
Epoch 4/50
8/8 [==============================] - 1s 187ms/step - loss: 0.2062 - val_loss: 0.2768

Epoch 00004: val_loss did not improve from 0.27591
Epoch 5/50
8/8 [==============================] - 1s 187ms/step - loss: 0.2055 - val_loss: 0.2770

Epoch 00005: val_loss did not improve from 0.27591
Epoch 6/50
8/8 [==============================] - 1s 186ms/step - loss: 0.2053 - val_loss: 0.2771

Epoch 00006: ReduceLROnPlateau reducing learning rate to 9.999999310821295e-05.

Epoch 00006: val_loss did not improve from 0.27591
Epoch 7/50
8/8 [===

In [ ]:
def predict(input_sentence,model):

  input_sequence=tokenizer_raw.texts_to_sequences([input_sentence])
  

  inputs=pad_sequences(input_sequence,maxlen=170,padding='post')
  inputs=tf.convert_to_tensor(inputs)
  result=''
  units=100
  hidden=[tf.zeros((1,units))]
  encoder_output,hidden_state,cell_state=model.encoder(inputs,hidden)
  dec_hidden=hidden_state
  dec_input=tf.expand_dims([tokenizer_std.word_index['\t']],0)
  for t in range(202):
      predictions,dec_hidden,cell_state,=model.layers[1](dec_input,[dec_hidden,cell_state])
      output=model.layers[2](predictions[0])
      predicted_id=tf.argmax(output[0]).numpy()
      result+=tokenizer_std.index_word[predicted_id]
      if tokenizer_std.word_index['\n']==predicted_id:
          return result
      dec_input= tf.expand_dims([predicted_id],0)
  return result

### Predictions

In [ ]:
input_sentence = tr.Source.iloc[1]
print(input_sentence)
print(tr.Target_out.iloc[1])
print(predict(input_sentence))

hey i think i will go cut lah. U think can get slot at 2 ?
Hey I think I will go and cut. You think can get slot at 2?

HIII o o o o o o o o o o o o o o o o o o o o o o o o o o o o o?Ta?T



In [ ]:
input_sentence = 'watch wat'
predict(input_sentence, model)

'WIhZ tae.Ya.Ya\n'

### Perplexity

In [ ]:
loss=model.evaluate([Source_inp_val, target_inp_val], target_out_val)

4/4 [==============================] - 0s 39ms/step - loss: 0.2777


In [ ]:
#https://towardsdatascience.com/perplexity-in-language-models-87a196019a94
perplexity=2**(loss)
perplexity

1.2122830038899854

### Bleu Score

In [ ]:
import nltk.translate.bleu_score as bleu
bleu_score=[]
for i in range(20):
  decoded_sent=predict(val.Source.values[i].split(), model)
  bleu_score.append(bleu.sentence_bleu(val.Target_out.values[i].split(), decoded_sent))

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [ ]:
avg_bleu_score=sum(bleu_score)/len(bleu_score)
avg_bleu_score

0.3746864939524901